In [1]:
import neo4j

import pandas as pd

from IPython.display import display

In [2]:
driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("neo4j","ucb_mids_w205"))

In [3]:
session = driver.session(database="neo4j")

In [4]:
def my_neo4j_wipe_out_database():
    "wipe out database by deleting all nodes and relationships"
    
    query = "match (node)-[relationship]->() delete node, relationship"
    session.run(query)
    
    query = "match (node) delete node"
    session.run(query)

In [5]:
def my_neo4j_run_query_pandas(query, **kwargs):
    "run a query and return the results in a pandas dataframe"
    
    result = session.run(query, **kwargs)
    
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    
    return df

In [6]:
def my_neo4j_nodes_relationships():
    "print all the nodes and relationships"
   
    print("-------------------------")
    print("  Nodes:")
    print("-------------------------")
    
    query = """
        match (n) 
        return n.name as node_name, labels(n) as labels
        order by n.name
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_nodes = df.shape[0]
    
    display(df)
    
    print("-------------------------")
    print("  Relationships:")
    print("-------------------------")
    
    query = """
        match (n1)-[r]->(n2) 
        return n1.name as node_name_1, labels(n1) as node_1_labels, 
            type(r) as relationship_type, n2.name as node_name_2, labels(n2) as node_2_labels
        order by node_name_1, node_name_2
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_relationships = df.shape[0]
    
    display(df)
    
    density = (2 * number_relationships) / (number_nodes * (number_nodes - 1))
    
    print("-------------------------")
    print("  Density:", f'{density:.1f}')
    print("-------------------------")
    

In [7]:
df = my_neo4j_nodes_relationships()

-------------------------
  Nodes:
-------------------------


,node_name,labels
0,#1 Coal Mine (Ki-Coal) M3353 Operating nan,[Energy_Project]
1,#1 Coal Mine (Knott County) M4627 Operating nan,[Energy_Project]
2,#1 Coal Mine M3354 Operating nan,[Energy_Project]
3,#1 Mine (KY) M4631 Operating nan,[Energy_Project]
4,#1 Strip Mine M3355 Operating nan,[Energy_Project]
...,...,...
38102,Şırnak Silopi (CİNER) power station Unit 1,[Energy_Project]
38103,Şırnak Silopi (CİNER) power station Unit 2,[Energy_Project]
38104,Şırnak Silopi (CİNER) power station Unit 3,[Energy_Project]
38105,Špitálka power station 1,[Energy_Project]


-------------------------
  Relationships:
-------------------------


,node_name_1,node_1_labels,relationship_type,node_name_2,node_2_labels
0,1590 Energy,[Entity],SHARE,Vivant,[Entity]
1,3 Main Mine 3M M4701 Operating nan,[Energy_Project],SHARE,Hwange Colliery,[Entity]
2,3-Energoortalyk,[Entity],SHARE,Eurasian Resources Group,[Entity]
3,3B Power,[Entity],SHARE,Chugoku Electric Power International Netherlands,[Entity]
4,3B Power,[Entity],SHARE,Mitsui & Co,[Entity]
...,...,...,...,...,...
33894,Şişecam Enerji,[Entity],SHARE,Türkiye İş Bankasi,[Entity]
33895,Şırnak Silopi (CİNER) power station Unit 1,[Energy_Project],SHARE,Silopi Elektrik Üretim,[Entity]
33896,Şırnak Silopi (CİNER) power station Unit 2,[Energy_Project],SHARE,Silopi Elektrik Üretim,[Entity]
33897,Şırnak Silopi (CİNER) power station Unit 3,[Energy_Project],SHARE,Silopi Elektrik Üretim,[Entity]


-------------------------
  Density: 0.0
-------------------------


# Harmonic Centrality

In [8]:
query = "CALL gds.graph.drop('ds_graph', false)"
session.run(query)

query = "CALL gds.graph.project('ds_graph', 'Entity', 'SHARE', {relationshipProperties: 'share'})"
session.run(query)

In [9]:
query = """

CALL gds.alpha.closeness.harmonic.stream('ds_graph', {})
YIELD nodeId, centrality
RETURN gds.util.asNode(nodeId).name AS name, centrality as closeness
ORDER BY centrality DESC

"""

my_neo4j_run_query_pandas(query)

,name,closeness
0,Blackrock,0.017444
1,The Vanguard Group,0.015932
2,Blackrock Advisors,0.012139
3,The Master Trust Bank of Japan,0.010784
4,Custody Bank of Japan,0.009399
...,...,...
12515,Üzülmez Taşkömürü Işletme Müessesesi,0.000000
12516,ČEZ Polska,0.000000
12517,İzdemir Enerji Elektrik Üretim,0.000000
12518,İÇDAŞ Elektrik Enerjisi Üretim ve Yatırım,0.000000


In [10]:
df = my_neo4j_run_query_pandas(query)

In [11]:
df.head(10)

,name,closeness
0,Blackrock,0.017444
1,The Vanguard Group,0.015932
2,Blackrock Advisors,0.012139
3,The Master Trust Bank of Japan,0.010784
4,Custody Bank of Japan,0.009399
5,China Energy Investment,0.008743
6,Nippon Life Insurance,0.007156
7,Meiji Yasuda Life Insurance,0.006956
8,Mitsubishi UFJ Trust and Banking,0.006916
9,The Norinchukin Trust & Banking,0.006916


# Betweenness Centrality

In [12]:
query = "CALL gds.graph.drop('ds_graph', false)"
session.run(query)

query = "CALL gds.graph.project('ds_graph', 'Entity', {SHARE: {properties: 'share'}})"
session.run(query)

In [13]:
query = """

CALL gds.betweenness.stream('ds_graph', {relationshipWeightProperty: 'share'})
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).name AS name, score as betweenness
ORDER BY betweenness DESC

"""

my_neo4j_run_query_pandas(query)

,name,betweenness
0,The Master Trust Bank of Japan,1525.0
1,Custody Bank of Japan,1174.0
2,Chongqing Energy Investment Group,809.0
3,Blackrock,655.0
4,Mitsui & Co,484.0
...,...,...
12515,İzdemir Enerji Elektrik Üretim,0.0
12516,İÇDAŞ,0.0
12517,İÇDAŞ Elektrik Enerjisi Üretim ve Yatırım,0.0
12518,Şişecam Enerji,0.0


In [14]:
df = my_neo4j_run_query_pandas(query)

In [15]:
df.head(10)

,name,betweenness
0,The Master Trust Bank of Japan,1525.0
1,Custody Bank of Japan,1174.0
2,Chongqing Energy Investment Group,809.0
3,Blackrock,655.0
4,Mitsui & Co,484.0
5,JERA,362.0
6,Marubeni,338.0
7,Glencore,336.0
8,Chongqing Energy Investment Runxin No. 1 Enter...,290.0
9,Mitsubishi,283.0
